<a href="https://colab.research.google.com/github/j-min/HiREST/blob/main/val_inference_and_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inference and Evaluation of moment retrieval / moment segmentation / step captioning tasks with our joint baseline model on HiREST val split

* Project page: https://hirest-cvpr2023.github.io
* Code: https://github.com/j-min/HiREST

In [ ]:
# Make sure GPU is used
!nvidia-smi

Wed Mar 29 20:00:06 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!git clone https://github.com/j-min/HiREST

In [ ]:
cd HiREST

# Install Packages

In [ ]:
!pip install allennlp_models # requires torch<1.13.0
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached allennlp_models-2.10.1-py3-none-any.whl (464 kB)
  Using cached datasets-2.11.0-py3-none-any.whl (468 kB)
  Using cached allennlp-2.10.1-py3-none-any.whl (730 kB)
  Using cached cached_path-1.1.6-py3-none-any.whl (26 kB)
  Using cached transformers-4.20.1-py3-none-any.whl (4.4 MB)
  Using cached wandb-0.12.21-py2.py3-none-any.whl (1.8 MB)
  Using cached spacy-3.3.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.3 MB)
  Using cached huggingface_hub-0.13.3-py3-none-any.whl (199 kB)
  Using cached aiohttp-3.8.4-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.0 MB)
  Using cached cached_path-1.1.5-py3-none-any.whl (26 kB)
  Using cached boto3-1.26.102-py3-none-any.whl (135 kB)
  Using cached cached_path-1.1.4-py3-none-any.whl (26 kB)
  Using cached cached_path-1.1.3-py3-none-any.whl (26 kB)
INFO: pip is looking at multiple versions of base58 to determin

In [ ]:
# Download coco evaluation utility files
import language_evaluation; language_evaluation.download('coco')

# Downloaded Pre-extracted features

In [ ]:
# ASR
!wget https://huggingface.co/j-min/HiREST-baseline/resolve/main/ASR.zip
# ASR_feats
!wget https://huggingface.co/j-min/HiREST-baseline/resolve/main/ASR_feats_all-MiniLM-L6-v2.zip
# eva_clip_feats
!wget https://huggingface.co/j-min/HiREST-baseline/resolve/main/eva_clip_features.zip

--2023-03-29 20:07:32--  https://huggingface.co/j-min/HiREST-baseline/resolve/main/ASR.zip
Resolving huggingface.co (huggingface.co)... 34.203.133.210, 54.82.45.103, 35.173.225.216, ...
Connecting to huggingface.co (huggingface.co)|34.203.133.210|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/43/e5/43e5477f8142d4d657272c2d83709064f52cea70061d7f463fe4e9973cc4707b/0b452d38e30064dc7273a58b7b73ec33e307ff83d30048a472777f56e3a29fbc?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27ASR.zip%3B+filename%3D%22ASR.zip%22%3B&response-content-type=application%2Fzip&Expires=1680379653&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzQzL2U1LzQzZTU0NzdmODE0MmQ0ZDY1NzI3MmMyZDgzNzA5MDY0ZjUyY2VhNzAwNjFkN2Y0NjNmZTRlOTk3M2NjNDcwN2IvMGI0NTJkMzhlMzAwNjRkYzcyNzNhNThiN2I3M2VjMzNlMzA3ZmY4M2QzMDA0OGE0NzI3NzdmNTZlM2EyOWZiYz9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSomcmVzcG9uc2UtY29udGVudC

In [ ]:
!unzip -q ASR.zip
!unzip -q ASR_feats_all-MiniLM-L6-v2.zip
!unzip -q eva_clip_features.zip

In [ ]:
!mv ASR/ data/ASR/
!mv ASR_feats_all-MiniLM-L6-v2/ data/ASR_feats_all-MiniLM-L6-v2/
!mv eva_clip_features/ data/eva_clip_features/

# !rm ASR.zip ASR_feats_all-MiniLM-L6-v2.zip eva_clip_features.zip

# Download Pretrained Weights

In [ ]:
!mkdir ./checkpoints/
!mkdir ./checkpoints/hirest_joint_model/
!mkdir ./pretrained_weights/

In [ ]:
# Pretrained CLIP4Caption
!wget https://huggingface.co/j-min/HiREST-baseline/resolve/main/clip4caption_vit-b-32_model.bin
!mv clip4caption_vit-b-32_model.bin ./pretrained_weights/clip4caption_vit-b-32_model.bin

# EVA-CLIP-G (frozen visual/textual encoder)
!wget https://huggingface.co/BAAI/EVA/resolve/main/eva_clip_psz14.pt
!mv eva_clip_psz14.pt ./pretrained_weights/eva_clip_psz14.pt

# Finetuned HiREST joint baseline model
!wget https://huggingface.co/j-min/HiREST-baseline/resolve/main/HiREST_BEST.pth
!mv HiREST_BEST.pth ./checkpoints/hirest_joint_model/BEST.pth

--2023-03-29 20:11:03--  https://huggingface.co/j-min/HiREST-baseline/resolve/main/clip4caption_vit-b-32_model.bin
Resolving huggingface.co (huggingface.co)... 54.82.45.103, 34.203.133.210, 52.2.178.255, ...
Connecting to huggingface.co (huggingface.co)|54.82.45.103|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/43/e5/43e5477f8142d4d657272c2d83709064f52cea70061d7f463fe4e9973cc4707b/e4814e953cb088c2f096ac346b003a750bed4f899b7af7bbd9513d43247837be?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27clip4caption_vit-b-32_model.bin%3B+filename%3D%22clip4caption_vit-b-32_model.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1680379864&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzQzL2U1LzQzZTU0NzdmODE0MmQ0ZDY1NzI3MmMyZDgzNzA5MDY0ZjUyY2VhNzAwNjFkN2Y0NjNmZTRlOTk3M2NjNDcwN2IvZTQ4MTRlOTUzY2IwODhjMmYwOTZhYzM0NmIwMDNhNzUwYmVkNGY4OTliN2FmN2JiZDk1MTNkN

# Run model inference

In [ ]:
!bash scripts/run_val.sh

Namespace(data_dir='./data/val_testing/', video_feature_dir='./data/eva_clip_features', asr_dir='./data/ASR', asr_feature_dir='./data/ASR_feats_all-MiniLM-L6-v2', task_moment_retrieval=True, task_moment_segmentation=True, task_step_captioning=True, use_gaussian_blur=False, train_batch_size=5, eval_batch_size=5, clip_grad_norm=5.0, gradient_accumulation_steps=1, epochs=50, lr=1e-05, weight_decay=0.0, warmup_steps=0.1, seed=2222, optim='adamw', fp16=False, train=False, end_to_end=False, n_model_frames=-1, num_workers=2, distributed=False, device='cuda', num_beams=3, load=None, ckpt_dir='./checkpoints/hirest_joint_model/', comment='', num_thread_reader=16, n_display=16, max_frames_step_captioning=20, max_words=48, visual_num_hidden_layers=2, decoder_num_hidden_layers=2, local_rank=0, gpu=0, world_size=1)
Process Launching at GPU 0
NLGEval not installed, if you want to train CLIP4Caption,
          please install it via pip install git+https://github.com/Maluuba/nlg-eval.git@master
tasks: 

# Calculate metrics

CLIPScore computation is skipped in this colab, as it requires video frame extraction for whole test videos. You can add `--frame_dir` to `evaluate.py`. 

In [ ]:
!bash scripts/score_val.bash

Moment Retrieval
Namespace(task='moment_retrieval', gt_data='./data/val_testing/all_data_test.json', pred_data='./checkpoints/hirest_joint_model//test_moment_retrieval_BEST.json', device=-1, print_per_category=False, help=False, preprocess_moment_bounds=False, replace_pred_moment_bounds=False, frame_dir='None')
100% 292/292 [00:00<00:00, 323833.10it/s]
100% 292/292 [00:00<00:00, 225011.35it/s]
{'total_videos': 193, 'R@0.5': 71.50259067357513, 'R@0.7': 37.30569948186528}
\n
Moment Segmentation
Namespace(task='moment_segmentation', gt_data='./data/val_testing/formated_moment_evaluation_gt.json', pred_data='./checkpoints/hirest_joint_model//test_moment_segmentation_BEST.json', device=-1, print_per_category=False, help=False, preprocess_moment_bounds=True, replace_pred_moment_bounds=False, frame_dir='None')
100% 78/78 [00:00<00:00, 933.22it/s]
100% 78/78 [00:00<00:00, 630.30it/s]
100% 78/78 [00:00<00:00, 580.78it/s]
{'recall': {'0.5': 36.20448704665185, '0.7': 16.44956170200275}, 'precisio